In [ ]:
# default_exp datasets.bases.ctr

# CTR Dataset
> Implementation of base modules for ctr dataset.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.nb_imports import *
from fastcore.test import *

In [ ]:
#export
from typing import Any, Iterable, List, Optional, Tuple, Union, Callable

import itertools
import numpy as np
import pandas as pd
import h5py
import pickle
import sklearn.preprocessing as sklearn_preprocess
from collections import Counter, OrderedDict, defaultdict
import io
import os
import logging
import json
from datetime import datetime, date

import torch
from torch.utils.data import DataLoader, Dataset

from pytorch_lightning import LightningDataModule

from recohut.datasets.bases.common import Dataset as BaseDataset
from recohut.utils.common_utils import download_url

In [ ]:
#exporti
class Tokenizer(object):
    def __init__(self, topk_words=None, na_value=None, min_freq=1, splitter=None, 
                 lower=False, oov_token=0, max_len=0, padding_type="pre"):
        self._topk_words = topk_words
        self._na_value = na_value
        self._min_freq = min_freq
        self._lower = lower
        self._splitter = splitter
        self.oov_token = oov_token # use 0 for __OOV__
        self.word_counts = Counter()
        self.vocab = dict()
        self.vocab_size = 0 # include oov and padding
        self.max_len = max_len
        self.padding_type = padding_type

    def fit_on_texts(self, texts, use_padding=True):
        tokens = list(texts)
        if self._splitter is not None: # for sequence
            text_splits = [text.split(self._splitter) for text in texts if not pd.isnull(text)]
            if self.max_len == 0:
                self.max_len = max(len(x) for x in text_splits)
            tokens = list(itertools.chain(*text_splits))
        if self._lower:
            tokens = [tk.lower() for tk in tokens]
        if self._na_value is not None:
            tokens = [tk for tk in tokens if tk != self._na_value]
        self.word_counts = Counter(tokens)
        words = [token for token, count in self.word_counts.most_common() if count >= self._min_freq]
        self.word_counts.clear() # empty the dict to save memory
        if self._topk_words:
            words = words[0:self._topk_words]
        self.vocab = dict((token, idx) for idx, token in enumerate(words, 1 + self.oov_token))
        self.vocab["__OOV__"] = self.oov_token
        if use_padding:
            self.vocab["__PAD__"] = len(words) + self.oov_token + 1 # use the last index for __PAD__
        self.vocab_size = len(self.vocab) + self.oov_token

    def encode_category(self, categories):
        category_indices = [self.vocab.get(x, self.oov_token) for x in categories]
        return np.array(category_indices)

    def encode_sequence(self, texts):
        sequence_list = []
        for text in texts:
            if pd.isnull(text) or text == '':
                sequence_list.append([])
            else:
                sequence_list.append([self.vocab.get(x, self.oov_token) for x in text.split(self._splitter)])
        sequence_list = self.padding(sequence_list, maxlen=self.max_len, value=self.vocab_size - 1,
                                padding=self.padding_type, truncating=self.padding_type)
        return np.array(sequence_list)
    
    def load_pretrained_embedding(self, feature_name, pretrain_path, embedding_dim, output_path):
        with h5py.File(pretrain_path, 'r') as hf:
            keys = hf["key"][:]
            pretrained_vocab = dict(zip(keys, range(len(keys))))
            pretrained_emb = hf["value"][:]
        embedding_matrix = np.random.normal(loc=0, scale=1.e-4, size=(self.vocab_size, embedding_dim))
        for word, idx in self.vocab.items():
            if word in pretrained_vocab:
                embedding_matrix[idx] = pretrained_emb[pretrained_vocab[word]]
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        with h5py.File(output_path, 'a') as hf:
            hf.create_dataset(feature_name, data=embedding_matrix)

    def set_vocab(self, vocab):
        self.vocab = vocab
        self.vocab_size = len(self.vocab) + self.oov_token

    @staticmethod
    def padding(sequences, maxlen=None, dtype='int32',
                padding='pre', truncating='pre', value=0.):
        """ Pads sequences (list of list) to the ndarray of same length """
        assert padding in ["pre", "post"], "Invalid padding={}.".format(padding)
        assert truncating in ["pre", "post"], "Invalid truncating={}.".format(truncating)
        
        if maxlen is None:
            maxlen = max(len(x) for x in sequences)
        arr = np.full((len(sequences), maxlen), value, dtype=dtype)
        for idx, x in enumerate(sequences):
            if len(x) == 0:
                continue  # empty list
            if truncating == 'pre':
                trunc = x[-maxlen:]
            else:
                trunc = x[:maxlen]
            trunc = np.asarray(trunc, dtype=dtype)

            if padding == 'pre':
                arr[idx, -len(trunc):] = trunc
            else:
                arr[idx, :len(trunc)] = trunc
        return arr

In [ ]:
#exporti
class Normalizer(object):
    def __init__(self, normalizer):
        if not callable(normalizer):
            self.callable = False
            if normalizer in ['StandardScaler', 'MinMaxScaler']:
                self.normalizer = getattr(sklearn_preprocess, normalizer)()
            else:
                raise NotImplementedError('normalizer={}'.format(normalizer))
        else:
            # normalizer is a method
            self.normalizer = normalizer
            self.callable = True

    def fit(self, X):
        if not self.callable:
            null_index = np.isnan(X)
            self.normalizer.fit(X[~null_index].reshape(-1, 1))

    def normalize(self, X):
        if self.callable:
            return self.normalizer(X)
        else:
            return self.normalizer.transform(X.reshape(-1, 1)).flatten()

In [ ]:
#exporti
class FeatureMap(object):
    def __init__(self, dataset_id='ctr'):
        self.dataset_id = dataset_id
        self.num_fields = 0
        self.num_features = 0
        self.feature_len = 0
        self.feature_specs = OrderedDict()
        
    def set_feature_index(self):
        logging.info("Set feature index...")
        idx = 0
        for feature, feature_spec in self.feature_specs.items():
            if feature_spec["type"] != "sequence":
                self.feature_specs[feature]["index"] = idx
                idx += 1
            else:
                seq_indexes = [i + idx for i in range(feature_spec["max_len"])]
                self.feature_specs[feature]["index"] = seq_indexes
                idx += feature_spec["max_len"]
        self.feature_len = idx

    def get_feature_index(self, feature_type=None):
        feature_indexes = []
        if feature_type is not None:
            if not isinstance(feature_type, list):
                feature_type = [feature_type]
            feature_indexes = [feature_spec["index"] for feature, feature_spec in self.feature_specs.items()
                               if feature_spec["type"] in feature_type]
        return feature_indexes

    def load(self, json_file):
        logging.info("Load feature_map from json: " + json_file)
        with io.open(json_file, "r", encoding="utf-8") as fd:
            feature_map = json.load(fd, object_pairs_hook=OrderedDict)
        if feature_map["dataset_id"] != self.dataset_id:
            raise RuntimeError("dataset_id={} does not match to feature_map!".format(self.dataset_id))
        self.num_fields = feature_map["num_fields"]
        self.num_features = feature_map.get("num_features", None)
        self.feature_len = feature_map.get("feature_len", None)
        self.feature_specs = OrderedDict(feature_map["feature_specs"])

    def save(self, json_file):
        logging.info("Save feature_map to json: " + json_file)
        if not os.path.exists(os.path.dirname(json_file)):
            os.makedirs(os.path.dirname(json_file))
        feature_map = OrderedDict()
        feature_map["dataset_id"] = self.dataset_id
        feature_map["num_fields"] = self.num_fields
        feature_map["num_features"] = self.num_features
        feature_map["feature_len"] = self.feature_len
        feature_map["feature_specs"] = self.feature_specs
        with open(json_file, "w") as fd:
            json.dump(feature_map, fd, indent=4)

In [ ]:
#export
class CTRDataset(torch.utils.data.Dataset, BaseDataset):
    feature_cols = list()
    label_col = dict()

    def __init__(self,
                 data_dir,
                 data_type=None,
                 *args,
                 **kwargs):
        super().__init__(data_dir)
        self.data_type = data_type
        self.pickle_file = os.path.join(self.processed_dir, "feature_encoder.pkl")
        self.json_file = os.path.join(self.processed_dir, "feature_map.json")
        self.feature_cols = self._complete_feature_cols(self.feature_cols)
        self.feature_map = FeatureMap()
        self.encoders = dict()

        if self.data_type == 'train':
            self.darray =  self.load_data(self.raw_paths[0])
            self.num_samples = len(self.darray)
        elif self.data_type == 'valid':
            self.darray = self.load_data(self.raw_paths[1])
            self.validation_samples = len(self.darray)
        elif self.data_type == 'test':
            self.darray = self.load_data(self.raw_paths[2])
            self.test_samples = len(self.darray)
        elif self.data_type is None:
            self._process()

    @property
    def raw_file_names(self):
        raise NotImplementedError

    @property
    def processed_file_names(self):
        return ['feature_encoder.pkl',
                'feature_map.json',
                'train_sample.h5',
                'valid_sample.h5',
                'test_sample.h5']

    def download(self):
        raise NotImplementedError

    def process(self):
        self.fit(self.raw_paths[0])

    @staticmethod
    def _complete_feature_cols(feature_cols):
        full_feature_cols = []
        for col in feature_cols:
            name_or_namelist = col["name"]
            if isinstance(name_or_namelist, list):
                for _name in name_or_namelist:
                    _col = col.copy()
                    _col["name"] = _name
                    full_feature_cols.append(_col)
            else:
                full_feature_cols.append(col)
        return full_feature_cols

    def read_csv(self, data_path):
        all_cols = self.feature_cols + [self.label_col]
        dtype_dict = dict((x["name"], eval(x["dtype"]) if isinstance(x["dtype"], str) else x["dtype"]) 
                          for x in all_cols)
        ddf = pd.read_csv(data_path, dtype=dtype_dict, memory_map=True) 
        return ddf

    def _preprocess(self, ddf):
        all_cols = [self.label_col] + self.feature_cols[::-1]
        for col in all_cols:
            name = col["name"]
            if name in ddf.columns and ddf[name].isnull().values.any():
                ddf[name] = self._fill_na(col, ddf[name])
            if "preprocess" in col and col["preprocess"] != "":
                preprocess_fn = getattr(self, col["preprocess"])
                ddf[name] = preprocess_fn(ddf, name)
        active_cols = [self.label_col["name"]] + [col["name"] for col in self.feature_cols if col["active"]]
        ddf = ddf.loc[:, active_cols]
        return ddf

    def _fill_na(self, col, series):
        na_value = col.get("na_value")
        if na_value is not None:
            return series.fillna(na_value)
        elif col["dtype"] == "str":
            return series.fillna("")
        else:
            raise RuntimeError("Feature column={} requires to assign na_value!".format(col["name"]))

    def fit(self, train_data, min_categr_count=1, num_buckets=10):           
        ddf = self.read_csv(train_data)
        ddf = self._preprocess(ddf)
        self.feature_map.num_fields = 0
        for col in self.feature_cols:
            if col["active"]:
                name = col["name"]
                self.fit_feature_col(col, ddf, 
                                     min_categr_count=min_categr_count,
                                     num_buckets=num_buckets)
                self.feature_map.num_fields += 1
        self.feature_map.set_feature_index()
        self.save_pickle(self.pickle_file)
        self.feature_map.save(self.json_file)
        
    def fit_feature_col(self, feature_column, ddf, min_categr_count=1, num_buckets=10):
        name = feature_column["name"]
        feature_type = feature_column["type"]
        feature_source = feature_column.get("source", "")
        self.feature_map.feature_specs[name] = {"source": feature_source,
                                                "type": feature_type}
        if "min_categr_count" in feature_column:
            min_categr_count = feature_column["min_categr_count"]
        self.feature_map.feature_specs[name]["min_categr_count"] = min_categr_count
        if "embedding_dim" in feature_column:
            self.feature_map.feature_specs[name]["embedding_dim"] = feature_column["embedding_dim"]
        feature_values = ddf[name].values
        if feature_type == "numeric":
            normalizer_name = feature_column.get("normalizer", None)
            if normalizer_name is not None:
                normalizer = Normalizer(normalizer_name)
                normalizer.fit(feature_values)
                self.encoders[name + "_normalizer"] = normalizer
            self.feature_map.num_features += 1
        elif feature_type == "categorical":
            encoder = feature_column.get("encoder", "")
            if encoder != "":
                self.feature_map.feature_specs[name]["encoder"] = encoder
            if encoder == "":
                tokenizer = Tokenizer(min_freq=min_categr_count, 
                                      na_value=feature_column.get("na_value", ""))
                if "share_embedding" in feature_column:
                    self.feature_map.feature_specs[name]["share_embedding"] = feature_column["share_embedding"]
                    tokenizer.set_vocab(self.encoders["{}_tokenizer".format(feature_column["share_embedding"])].vocab)
                else:
                    if self.is_share_embedding_with_sequence(name):
                        tokenizer.fit_on_texts(feature_values, use_padding=True)
                        self.feature_map.feature_specs[name]["padding_idx"] = tokenizer.vocab_size - 1
                    else:
                        tokenizer.fit_on_texts(feature_values, use_padding=False)
                self.encoders[name + "_tokenizer"] = tokenizer
                self.feature_map.num_features += tokenizer.vocab_size
                self.feature_map.feature_specs[name]["vocab_size"] = tokenizer.vocab_size
                if "pretrained_emb" in feature_column:
                    self.feature_map.feature_specs[name]["pretrained_emb"] = "pretrained_embedding.h5"
                    self.feature_map.feature_specs[name]["freeze_emb"] = feature_column.get("freeze_emb", True)
                    tokenizer.load_pretrained_embedding(name,
                                                        feature_column["pretrained_emb"], 
                                                        feature_column["embedding_dim"],
                                                        os.path.join(self.processed_dir, "pretrained_embedding.h5"))
            elif encoder == "numeric_bucket":
                num_buckets = feature_column.get("num_buckets", num_buckets)
                qtf = sklearn_preprocess.QuantileTransformer(n_quantiles=num_buckets + 1)
                qtf.fit(feature_values)
                boundaries = qtf.quantiles_[1:-1]
                self.feature_map.feature_specs[name]["vocab_size"] = num_buckets
                self.feature_map.num_features += num_buckets
                self.encoders[name + "_boundaries"] = boundaries
            elif encoder == "hash_bucket":
                num_buckets = feature_column.get("num_buckets", num_buckets)
                uniques = Counter(feature_values)
                num_buckets = min(num_buckets, len(uniques))
                self.feature_map.feature_specs[name]["vocab_size"] = num_buckets
                self.feature_map.num_features += num_buckets
                self.encoders[name + "_num_buckets"] = num_buckets
        elif feature_type == "sequence":
            encoder = feature_column.get("encoder", "MaskedAveragePooling")
            splitter = feature_column.get("splitter", " ")
            na_value = feature_column.get("na_value", "")
            max_len = feature_column.get("max_len", 0)
            padding = feature_column.get("padding", "post")
            tokenizer = Tokenizer(min_freq=min_categr_count, splitter=splitter, 
                                  na_value=na_value, max_len=max_len, padding=padding)
            if "share_embedding" in feature_column:
                self.feature_map.feature_specs[name]["share_embedding"] = feature_column["share_embedding"]
                tokenizer.set_vocab(self.encoders["{}_tokenizer".format(feature_column["share_embedding"])].vocab)
            else:
                tokenizer.fit_on_texts(feature_values, use_padding=True)
            self.encoders[name + "_tokenizer"] = tokenizer
            self.feature_map.num_features += tokenizer.vocab_size
            self.feature_map.feature_specs[name].update({"encoder": encoder,
                                                         "padding_idx": tokenizer.vocab_size - 1,
                                                         "vocab_size": tokenizer.vocab_size,
                                                         "max_len": tokenizer.max_len})
            if "pretrained_emb" in feature_column:
                self.feature_map.feature_specs[name]["pretrained_emb"] = "pretrained_embedding.h5"
                self.feature_map.feature_specs[name]["freeze_emb"] = feature_column.get("freeze_emb", True)
                tokenizer.load_pretrained_embedding(name,
                                                    feature_column["pretrained_emb"], 
                                                    feature_column["embedding_dim"],
                                                    os.path.join(self.processed_dir, "pretrained_embedding.h5"))
        else:
            raise NotImplementedError("feature_col={}".format(feature_column))

    def transform(self, ddf):
        ddf = self._preprocess(ddf)
        data_arrays = []
        for feature, feature_spec in self.feature_map.feature_specs.items():
            feature_type = feature_spec["type"]
            if feature_type == "numeric":
                numeric_array = ddf.loc[:, feature].fillna(0).apply(lambda x: float(x)).values
                normalizer = self.encoders.get(feature + "_normalizer")
                if normalizer:
                     numeric_array = normalizer.normalize(numeric_array)
                data_arrays.append(numeric_array) 
            elif feature_type == "categorical":
                encoder = feature_spec.get("encoder", "")
                if encoder == "":
                    data_arrays.append(self.encoders.get(feature + "_tokenizer") \
                                                    .encode_category(ddf.loc[:, feature].values))
                elif encoder == "numeric_bucket":
                    raise NotImplementedError
                elif encoder == "hash_bucket":
                    raise NotImplementedError
            elif feature_type == "sequence":
                data_arrays.append(self.encoders.get(feature + "_tokenizer") \
                                                .encode_sequence(ddf.loc[:, feature].values))
        label_name = self.label_col["name"]
        if ddf[label_name].dtype != np.float64:
            ddf.loc[:, label_name] = ddf.loc[:, label_name].apply(lambda x: float(x))
        data_arrays.append(ddf.loc[:, label_name].values) # add the label column at last
        data_arrays = [item.reshape(-1, 1) if item.ndim == 1 else item for item in data_arrays]
        data_array = np.hstack(data_arrays)
        return data_array

    def is_share_embedding_with_sequence(self, feature):
        for col in self.feature_cols:
            if col.get("share_embedding", None) == feature and col["type"] == "sequence":
                return True
        return False

    def load_pickle(self, pickle_file=None):
        return pickle.load(open(pickle_file, "rb"))

    def save_pickle(self, pickle_file):
        if not os.path.exists(os.path.dirname(pickle_file)):
            os.makedirs(os.path.dirname(pickle_file))
        pickle.dump(self.encoders, open(pickle_file, "wb"))
        
    def load_json(self, json_file):
        self.feature_map.load(json_file)

    def load_data(self, data_path, use_hdf5=True, data_format='csv'):
        self.load_json(self.json_file)
        self.encoders = self.load_pickle(self.pickle_file)
        if data_format == 'h5':
            data_array = self.load_hdf5(data_path)
            return data_array
        elif data_format == 'csv':
            hdf5_file = os.path.join(self.processed_dir, 
                                     os.path.splitext(os.path.basename(data_path))[0] + '.h5')
            if use_hdf5 and os.path.exists(hdf5_file):
                try:
                    data_array = self.load_hdf5(hdf5_file)
                    return data_array
                except:
                    print('Loading h5 file failed, reloading from {}'.format(data_path))
            ddf = self.read_csv(data_path)
            data_array = self.transform(ddf)
            if use_hdf5:
                self.save_hdf5(data_array, hdf5_file)
        return data_array

    def save_hdf5(self, data_array, data_path, key="data"):
        if not os.path.exists(os.path.dirname(data_path)):
            os.makedirs(os.path.dirname(data_path))
        with h5py.File(data_path, 'w') as hf:
            hf.create_dataset(key, data=data_array)

    def load_hdf5(self, data_path, key="data"):
        with h5py.File(data_path, 'r') as hf:
            data_array = hf[key][:]
        return data_array

    def __getitem__(self, index):
        X = self.darray[index, 0:-1]
        y = self.darray[index, -1]
        return X, y
    
    def __len__(self):
        return self.darray.shape[0]

In [ ]:
#export
class CTRDataModule(LightningDataModule):

    dataset_cls: str = ""

    def __init__(self,
                 data_dir: Optional[str] = None,
                 num_workers: int = 0,
                 normalize: bool = False,
                 batch_size: int = 32,
                 shuffle: bool = True,
                 pin_memory: bool = True,
                 drop_last: bool = False,
                 *args, 
                 **kwargs) -> None:
        """
        Args:
            data_dir: Where to save/load the data
            num_workers: How many workers to use for loading data
            normalize: If true applies rating normalize
            batch_size: How many samples per batch to load
            shuffle: If true shuffles the train data every epoch
            pin_memory: If true, the data loader will copy Tensors into CUDA pinned memory before
                        returning them
            drop_last: If true drops the last incomplete batch
        """
        super().__init__(data_dir)

        self.data_dir = data_dir if data_dir is not None else os.getcwd()
        self.num_workers = num_workers
        self.normalize = normalize
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.pin_memory = pin_memory
        self.drop_last = drop_last
        self.kwargs = kwargs

    def prepare_data(self, *args: Any, **kwargs: Any) -> None:
        """Saves files to data_dir."""
        self.dataset = self.dataset_cls(self.data_dir, **self.kwargs)

    def setup(self, stage: Optional[str] = None) -> None:
        """Creates train, val, and test dataset."""
        if stage == "fit" or stage is None:
            self.dataset_train = self.dataset_cls(self.data_dir, data_type='train', **self.kwargs)
            self.dataset_val = self.dataset_cls(self.data_dir, data_type='valid', **self.kwargs)
        if stage == "test" or stage is None:
            self.dataset_test = self.dataset_cls(self.data_dir, data_type='test', **self.kwargs)

    def train_dataloader(self, *args: Any, **kwargs: Any) -> DataLoader:
        """The train dataloader."""
        return self._data_loader(self.dataset_train, shuffle=self.shuffle)

    def val_dataloader(self, *args: Any, **kwargs: Any) -> Union[DataLoader, List[DataLoader]]:
        """The val dataloader."""
        return self._data_loader(self.dataset_val)

    def test_dataloader(self, *args: Any, **kwargs: Any) -> Union[DataLoader, List[DataLoader]]:
        """The test dataloader."""
        return self._data_loader(self.dataset_test)

    def _data_loader(self, dataset: Dataset, shuffle: bool = False) -> DataLoader:
        return DataLoader(
            dataset,
            batch_size=self.batch_size,
            shuffle=shuffle,
            num_workers=self.num_workers,
            drop_last=self.drop_last,
            pin_memory=self.pin_memory,
        )

Example

In [ ]:
class TaobaoDataset(CTRDataset):

    feature_cols = [{'name': ["userid","adgroup_id","pid","cate_id","campaign_id","customer","brand","cms_segid",
                                "cms_group_id","final_gender_code","age_level","pvalue_level","shopping_level","occupation"],
                        'active': True, 'dtype': 'str', 'type': 'categorical'}]
                        
    label_col = {'name': 'clk', 'dtype': float}

    train_url = "https://github.com/RecoHut-Datasets/sample_ctr/raw/v1/train_sample.csv"
    valid_url = "https://github.com/RecoHut-Datasets/sample_ctr/raw/v1/valid_sample.csv"
    test_url = "https://github.com/RecoHut-Datasets/sample_ctr/raw/v1/test_sample.csv"

    @property
    def raw_file_names(self):
        return ['train_sample.csv',
                'valid_sample.csv',
                'test_sample.csv']

    def download(self):
        download_url(self.train_url, self.raw_dir)
        download_url(self.valid_url, self.raw_dir)
        download_url(self.test_url, self.raw_dir)

    def convert_hour(self, df, col_name):
        return df['time_stamp'].apply(lambda ts: ts[11:13])

    def convert_weekday(self, df, col_name):
        def _convert_weekday(timestamp):
            dt = date(int(timestamp[0:4]), int(timestamp[5:7]), int(timestamp[8:10]))
            return dt.strftime('%w')
        return df['time_stamp'].apply(_convert_weekday)

    def convert_weekend(self, df, col_name):
        def _convert_weekend(timestamp):
            dt = date(int(timestamp[0:4]), int(timestamp[5:7]), int(timestamp[8:10]))
            return '1' if dt.strftime('%w') in ['6', '0'] else '0'
        return df['time_stamp'].apply(_convert_weekend)

In [ ]:
class TaobaoDataModule(CTRDataModule):
    dataset_cls = TaobaoDataset

In [ ]:
params = {'model_id': 'DCN_demo',
          'data_dir': '/content/data',
          'model_root': './checkpoints/',
          'dnn_hidden_units': [64, 64],
          'dnn_activations': "relu",
          'crossing_layers': 3,
          'learning_rate': 1e-3,
          'net_dropout': 0,
          'batch_norm': False,
          'optimizer': 'adamw',
          'task': 'binary_classification',
          'loss': 'binary_crossentropy',
          'metrics': ['logloss', 'AUC'],
          'embedding_dim': 10,
          'batch_size': 64,
          'epochs': 3,
          'shuffle': True,
          'seed': 2019,
          'use_hdf5': True,
          'workers': 1,
          'verbose': 0}

In [ ]:
ds = TaobaoDataModule(**params)
ds.prepare_data()
ds.setup()

for batch in ds.train_dataloader():
    print(batch)
    break

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:74: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  "DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7."
Processing...
Done!


[tensor([[ 9., 30.,  1., 24., 30., 30., 24.,  2.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 7., 22.,  2., 12., 22., 22.,  0.,  4.,  3.,  1.,  4.,  1.,  1.,  1.],
        [20., 96.,  1., 18., 94., 93., 63.,  9.,  9.,  2.,  2.,  2.,  1.,  1.],
        [15., 45.,  2., 15., 44., 44.,  0.,  6.,  7.,  1.,  5.,  1.,  1.,  1.],
        [11., 32.,  2., 25., 32., 32., 26.,  2.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 2.,  8.,  1., 14.,  9., 10., 14.,  1.,  2.,  1.,  2.,  0.,  1.,  1.],
        [ 5., 43.,  1.,  1., 42., 42., 35.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  2.,  1.,  8.,  4.,  5.,  9.,  1.,  1.,  1.,  1.,  0.,  1.,  1.],
        [18., 78.,  1., 43., 76., 75.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  1.],
        [ 3., 92.,  1.,  1., 90., 89.,  0.,  1.,  4.,  1.,  3.,  0.,  1.,  1.],
        [15., 59.,  2., 10., 57., 57., 44.,  6.,  7.,  1.,  5.,  1.,  1.,  1.],
        [ 5., 52.,  1.,  1., 51., 51.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [17., 90.,  1., 19., 88., 87., 

> **References**
> - FuxiCTR Project, https://github.com/xue-pai/FuxiCTR.

In [ ]:
#hide
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d -p recohut,pytorch_lightning

Author: Sparsh A.

Last updated: 2022-01-11 13:21:07

recohut          : 0.0.11
pytorch_lightning: 1.5.8

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.144+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

json   : 2.0.9
logging: 0.5.1.2
h5py   : 3.1.0
IPython: 5.5.0
numpy  : 1.19.5
six    : 1.15.0
sklearn: 0.0
torch  : 1.10.0+cu111
pandas : 1.1.5

